In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import os
import time

In [2]:
%%time 

# Abriendo los archivos .csv
folderpath =  r"C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data"
print("Files and directories in a specified path:")
files = []
filenames = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    filenames.append(filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_programas_sociales_hogar.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE223132.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE516171.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE758081.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC84DV.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC91.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC93DVdisciplina.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC95.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RECH23.csv
Wall time: 2.99 ms


In [456]:
for i in range(len(filenames)):
    filenames[i] = filenames[i][slice(0,-4)]
filenames

['endes_2021_programas_sociales_hogar',
 'endes_2021_RE223132',
 'endes_2021_RE516171',
 'endes_2021_RE758081',
 'endes_2021_REC84DV',
 'endes_2021_REC91',
 'endes_2021_REC93DVdisciplina',
 'endes_2021_REC95',
 'endes_2021_RECH23']

In [179]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51217/status,
Dashboard: http://127.0.0.1:51217/status,Workers: 4
Total threads: 8,Total memory: 7.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51218,Workers: 4
Dashboard: http://127.0.0.1:51217/status,Total threads: 8
Started: Just now,Total memory: 7.81 GiB
Comm: tcp://127.0.0.1:51245,Total threads: 2
Dashboard: http://127.0.0.1:51248/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:51221,


In [180]:
import dask.dataframe as dd

In [9]:
#asignamos nombres a los dataframes
programas_sociales = dd.read_csv(files[0], dtype = str)
nacimiento = dd.read_csv(files[1], dtype = str)
nupcialidad_fecundidad = dd.read_csv(files[2], dtype = str)
sida_condon = dd.read_csv(files[3], dtype = str)
mortalidad_violencia = dd.read_csv(files[4], dtype = str)
MEF = dd.read_csv(files[5], dtype = str) 
disciplina_infantil = dd.read_csv(files[6], dtype = str)
inmunizacion_salud = dd.read_csv(files[7], dtype = str)
vivienda = dd.read_csv(files[8], dtype = str)

In [33]:
# identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud


In [39]:
base_total = nacimiento.merge(nupcialidad_fecundidad, on = ['CASEID', 'ID1'], how ='left').merge(sida_condon, on = ['CASEID', 'ID1'], how ='left').merge(mortalidad_violencia, on = ['CASEID', 'ID1'], how ='left').merge(MEF, on = ['CASEID', 'ID1'], how ='left').merge(disciplina_infantil, on = ['CASEID', 'ID1'], how ='left').merge(inmunizacion_salud, on = ['CASEID', 'ID1'], how ='left')

In [41]:
base_total.compute()

,ID1,CASEID,V201,V202,V203,V204,V205,V206,V207,V208,...,S466DC11,S466MC11,S466YC11,S466WC11,S466HC11,S466DC12,S466MC12,S466YC12,S466WC12,S466HC12
0,2021,000100301 2,1,1,0,0,0,0,0,1,...,,,,,,,,,,
1,2021,000100301 4,1,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,000100401 1,6,4,0,0,2,0,0,1,...,10,3,2020,8640,725,26,5,2020,9010,750
3,2021,000100801 2,4,1,3,0,0,0,0,1,...,,,,,,,,,,
4,2021,000100801 4,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43132,2021,325410601 1,7,4,3,0,0,0,0,2,...,,,,,,,,,,
43133,2021,325410601 1,7,4,3,0,0,0,0,2,...,21,7,2019,9950,790,29,10,2019,11300,860
43134,2021,325410601 2,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43135,2021,325410801 2,2,1,1,0,0,0,0,1,...,,,,,,,,,,


In [172]:
df_final = base_total.compute()

In [173]:
df_final.dtypes

ID1         object
CASEID      object
V201        object
V202        object
V203        object
             ...  
S466DC12    object
S466MC12    object
S466YC12    object
S466WC12    object
S466HC12    object
Length: 1272, dtype: object

In [174]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = df_final[variables]
data_final.shape

Wall time: 619 ms


(43137, 27)

In [175]:
for i in variables:
        if data_final[i].dtype == 'O':
            data_final[i] = data_final[i].str.replace(" ", "")
            # Reemplazando los missing values por NaN
            data_final[i] = data_final[i].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            data_final[i] =  np.floor(pd.to_numeric(data_final[i], errors='coerce').astype('Int64'))

In [176]:
data_final.dtypes

ID1        Float64
SREGION    Float64
SPROVIN    Float64
SDISTRI    Float64
CASEID     Float64
S108N      Float64
S119       Float64
S119D      Float64
V218       Float64
V213       Float64
V225       Float64
S45D4      Float64
S45D5      Float64
S45P4      Float64
S45P5      Float64
S45B0      Float64
S45B1      Float64
S45B2      Float64
S45B3      Float64
S45F1      Float64
S45F2      Float64
V501       Float64
V714       Float64
D104       Float64
D106       Float64
D107       Float64
D108       Float64
dtype: object

In [177]:
# Agregando los datos para evaluar cuántas mujeres han sido víctimas de violencia en al menos una de sus modalidades
data_final['violencia'] = data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108']

# Contabilizando lo encontrado
data_final['violencia'].value_counts()

0.0    18395
1.0     4372
2.0     2621
3.0     1598
4.0      817
Name: violencia, dtype: Int64

In [178]:
# Creando variable dummy 

data_final['y'] = data_final['violencia']
data_final.loc[(data_final['y'] > 0), 'y'] = 1
data_final['y'].value_counts()

0.0    18395
1.0     9408
Name: y, dtype: Int64